# Housing Prediction

- Run a hyperparameter tuning while training a model

- Log every Hyperparameter and metrics in the MLFLOW UI

- Compare the result of the various runs in the MLFLOW UI

- Choose the best run and register it as a model

In [4]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

housing_data = fetch_california_housing()
print(housing_data)



{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]], shape=(20640, 8)), 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894], shape=(20640,)), 'frame': None, 'target_names': ['MedHouseVal'], 'feature_names': ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude'], 'DESCR': '.. _california_housing_dataset

In [5]:
df_data = pd.DataFrame(housing_data.data, columns=housing_data.feature_names)
df_data['price']= housing_data.target

# Train Test split, Model Hyperparameter Tuning and MLFLOW Experiments

In [11]:
from urllib.parse import urlparse
## Independent and Dependent features
X = df_data.drop(columns=['price'])
y = df_data['price']


In [21]:
## Hyperparameter tuning using GridSearchCV

def hyperparameter_tuning(X_train, y_train, param_grid):
    rf = RandomForestRegressor()
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, 
                               scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    return grid_search

In [24]:
## Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

from mlflow.models import infer_signature
signature= infer_signature(X_train, y_train)

#Define hyperparameter grid
param_grid = {
    'n_estimators' : [100, 200],          
    'max_depth' : [5, 10, None],
    'min_samples_split' : [2, 5],
    'min_samples_leaf' :[1, 2]
    }

## start the MLFLOW Experiments

with mlflow.start_run():
    ## Perform hyperparameter tuning
    grid_search = hyperparameter_tuning(X_train, y_train, param_grid)

    ## Get the best model 
    best_model = grid_search.best_estimator_

    ## Evaluate the best model 
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    ##log best parameters and metrics
    mlflow.log_param("best_n_estimators", grid_search.best_params_['n_estimators'])
    mlflow.log_param("best_max_depth", grid_search.best_params_['max_depth'])
    mlflow.log_param("best_min_samples_split", grid_search.best_params_['min_samples_split'])
    mlflow.log_param("best_min_samples_leaf", grid_search.best_params_['min_samples_leaf'])
    mlflow.log_metric("mse", mse)

    #Set tracking uri
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_url_type_store != "file":
        mlflow.sklearn.log_model(best_model, "model", registered_model_name="Best RandomForsert Model")
    else:
        mlflow.sklearn.log_model(best_model, "model", signature=signature)

    print("Best Model parameters: ", grid_search.best_params_)
    print("Mean Squared Error: ", mse)
    


Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   2.0s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   2.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   2.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   4.2s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   4.3s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   4.3s
[CV] END max_depth=5, min_samples_leaf=

2025/12/09 17:29:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Successfully registered model 'Best RandomForsert Model'.
2025/12/09 17:29:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best RandomForsert Model, version 1


Best Model parameters:  {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Mean Squared Error:  0.24672313002039759
🏃 View run defiant-dove-929 at: http://127.0.0.1:5000/#/experiments/0/runs/25c5bed55eae434fa15aa028f4445ba3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


Created version '1' of model 'Best RandomForsert Model'.
